# 🚀 Mapperatorinator LoRA Training

Notebook này được thiết kế để bạn có thể training model Mapperatorinator bằng phương pháp LoRA trên Google Colab GPU.

### 📝 Quy trình thực hiện:
1. **Chuẩn bị dữ liệu**: Đảm bảo bạn đã upload file `v30_dataset.zip` lên Google Drive.
2. **Cấu hình Runtime**: Chọn **Runtime -> Change runtime type -> GPU -> T4 GPU**.
3. **Chạy từng cell**: Nhấn nút Play (▶️) ở cạnh trái mỗi ô code.

---

## Bước 1: Kết nối Google Drive và chuẩn bị môi trường

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Clone repository
!git clone https://github.com/TheBlueSkyARL/Mapperatorinator.git -b test
%cd Mapperatorinator

# Cài đặt các thư viện cần thiết
!pip install -r requirements.txt
!pip install peft accelerate huggingface_hub

# Vá lỗi ValueError và config LoRA trong train.py
!sed -i 's/lora_config = LoraConfig(\**OmegaConf.to_object(args.lora))/lora_config = LoraConfig(\**args.lora)/g' osuT5/train.py

## Bước 2: Đăng nhập Hugging Face
Sử dụng Token loại **Read** từ [Hugging Face Settings](https://huggingface.co/settings/tokens).

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Bước 3: Giải nén v30_dataset từ Google Drive
*Lưu ý: MMRS dataset yêu cầu file metadata.parquet nằm cùng cấp với thư mục data.*

In [ ]:
!mkdir -p datasets

# Sửa lại đường dẫn này cho đúng với file v30_dataset.zip trên Drive của bạn
PATH_TO_ZIP = '/content/drive/MyDrive/v30_dataset.zip' 

import os
if os.path.exists(PATH_TO_ZIP):
    !unzip -o {PATH_TO_ZIP} -d datasets/v30_dataset
    print("✅ Đã giải nén v30_dataset thành công!")
else:
    print("❌ Không tìm thấy file zip tại: ", PATH_TO_ZIP)
    print("Hãy đảm bảo bạn đã upload file v30_dataset.zip lên Google Drive.")

## Bước 4: Bắt đầu Training LoRA (MMRS Mode)

In [ ]:
# Chạy lệnh training với định dạng mmrs và dataset v30
!python osuT5/train.py -cn lora \
    pretrained_path="OliBomby/Mapperatorinator-v30" \
    data.dataset_type="mmrs" \
    data.train_dataset_path="datasets/v30_dataset" \
    data.test_dataset_path="datasets/v30_dataset" \
    data.train_dataset_start=0 \
    data.train_dataset_end=1500 \
    data.test_dataset_start=1500 \
    data.test_dataset_end=1597 \
    dataloader.num_workers=2 \
    optim.batch_size=8 \
    optim.grad_acc=4 \
    optim.total_steps=2000 \
    precision="fp16" \
    compile=False

## ✅ Hoàn tất!
Trọng số sẽ được lưu trong `outputs/`. Bạn nên tải về Drive hoặc máy cá nhân sau khi hoàn tất.